In [259]:
import pandas as pd
import numpy as np

import scipy.stats as stats
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
plt.style.use('seaborn')

import urllib.request
from bs4 import BeautifulSoup

In [2]:
url = 'http://www1.nyc.gov/assets/ccrb/downloads/excel/ccrb_datatransparencyinitiative_20170207.xlsx'

# Read in all sheets of Excel file
xl = pd.read_excel(url, sheet_name=None)

# Print the sheetnames
xl.keys()

odict_keys(['Data Notes', 'Complaints_Allegations', 'Sheet3'])

In [6]:
xl['Data Notes'].iloc[0,0]

'The "Complaints_Allegations" sheet of this excel file contains data on all CCRB jurisdiction complaints closed in or after 2006.\nSome data in the Data Transparency Initiative reports on complaints received.\n\nExcel Column Descriptions:\n\nColumn A: DateStamp - the date the file was created.\nColumn B: UniqueComplaintId - a randomly assigned number used to uniquely identify distinct complaints. A single UniqueComplaintId may have multiple associated allegations.\nColumn C: Close Year - the year the complaint was closed by the CCRB.\nColumn D: CCRB Received Year - the year the complaint was received by the CCRB.\nColumn E: Borough of Occurrence - the borough in which the incident occurred.\nColumn F: Is Full Investigation - a logical marker indicating whether the complaint was fully investigated by the CCRB.\nColumn G: Complaint Has Video Evidence - a logical marker indicating whether the CCRB has collected any video associated with the complaint.\nColumn H: Complaint Filed Mode - how

In [11]:
data = xl['Complaints_Allegations']

In [368]:
data.sample(3)

,DateStamp,UniqueComplaintId,Close Year,Received Year,Borough of Occurrence,Is Full Investigation,Complaint Has Video Evidence,Complaint Filed Mode,Complaint Filed Place,Complaint Contains Stop & Frisk Allegations,Incident Location,Incident Year,Encounter Outcome,Reason For Initial Contact,Allegation FADO Type,Allegation Description
153405,2017-02-07,43302,2013,2013,Queens,False,False,Phone,IAB,False,Street/highway,2013,Arrest,C/V intervened on behalf of/observed encounter...,Force,Physical force
29762,2017-02-07,22278,2007,2006,Staten Island,True,False,Phone,CCRB,False,Street/highway,2005,Arrest,C/V at PCT to file complaint of crime,Force,Gun Pointed
32005,2017-02-07,28540,2007,2007,Brooklyn,True,False,Mail,CCRB,True,Apartment/house,2007,No Arrest or Summons,Report-domestic dispute,Abuse of Authority,Search (of person)


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206718 entries, 0 to 206717
Data columns (total 16 columns):
DateStamp                                      206718 non-null datetime64[ns]
UniqueComplaintId                              206718 non-null int64
Close Year                                     206718 non-null int64
Received Year                                  206718 non-null int64
Borough of Occurrence                          206216 non-null object
Is Full Investigation                          206718 non-null bool
Complaint Has Video Evidence                   206718 non-null bool
Complaint Filed Mode                           206718 non-null object
Complaint Filed Place                          206718 non-null object
Complaint Contains Stop & Frisk Allegations    206718 non-null bool
Incident Location                              203349 non-null object
Incident Year                                  206718 non-null int64
Encounter Outcome                              2067

* How many unique complaints (identified by 'UniqueComplaintId') with complete information (i.e. there are no missing values) appear in the dataset? 

In [18]:
# Get the number of unique complaints after removing rows with missing data:
data.dropna(axis=0).UniqueComplaintId.unique().shape

(68467,)

* What proportion of complaints occur in the borough with the largest number of complaints? For this question, only consider unique complaints with complete information. 

In [38]:
# Create a new dataframe with unique complaints and complete observations only
data_uni_complete = data.dropna(axis=0).drop_duplicates('UniqueComplaintId')

In [39]:
# Identify the borough with the largest number of complaints
data_uni_complete['Borough of Occurrence'].value_counts()

Brooklyn         23369
Manhattan        16067
Bronx            15603
Queens           10608
Staten Island     2763
Outside NYC         57
Name: Borough of Occurrence, dtype: int64

In [41]:
# Compute the % of complaints that occured in the borough with the largest n of complaints
data_uni_complete['Borough of Occurrence'].value_counts()[0] / data_uni_complete.shape[0]

0.3413177151036266

* How many complaints per 100k residents were there in the borough with the highest number of complaints per capita resulting from incidents in 2016? Find the 2016 population estimates of each borough on Wikipedia. Ignore complaints from "Outside NYC". For this question, only consider unique complaints with complete information. 

In [44]:
# Define 2016 population estimates for boroughs 
# Source: https://en.wikipedia.org/wiki/Demographics_of_New_York_City
mnhttn2016, brkln2016, queens2016, brnx2016, sttn2016, tot2016 = \
[1643734, 2629150, 2333054, 1455720, 476015, 8537673]

In [45]:
# A quick sanity check
mnhttn2016 + brkln2016 + queens2016 + brnx2016 + sttn2016 == tot2016

True

In [56]:
data_uni_complete[data_uni_complete['Incident Year'] == 2016]\
['Borough of Occurrence'].value_counts()

Brooklyn         1068
Manhattan         844
Bronx             772
Queens            589
Staten Island     149
Outside NYC         3
Name: Borough of Occurrence, dtype: int64

In [57]:
# Create a list of n of complaints per borough (2016) ignoring 'Outside NYC':
cmplnts2016 = \
data_uni_complete[data_uni_complete['Incident Year'] == 2016]\
['Borough of Occurrence'].value_counts()[:5]

In [67]:
# Create a list of boroughs names 
names = data_uni_complete[data_uni_complete['Incident Year'] == 2016]\
['Borough of Occurrence'].value_counts()[:5].index

In [59]:
# Create a list of populations
popul2016 = [brkln2016, mnhttn2016, brnx2016, queens2016, sttn2016]

In [72]:
# Identify the borough with the highest n of cmplnts per capita
for name, c, p in zip(names, cmplnts2016, popul2016):
    print(f'{name:15} {c/p}')

Brooklyn        0.00040621493638628455
Manhattan       0.0005134650740326598
Bronx           0.0005303217651746215
Queens          0.00025245879435280966
Staten Island   0.00031301534615505816


In [73]:
# Bronx had got the highest number of cmplnts per capita in 2016
# How many complaints per 100k residents in 2016 were there in Brnx?
0.0005303217651746215 * 100000

53.032176517462155

* What is the average number of years it takes for a complaint to be closed? For this question, only consider unique complaints with complete information. 

In [78]:
(data_uni_complete['Close Year'] - data_uni_complete['Received Year']).sum() / \
data_uni_complete.shape[0]

0.4743745161902814

* Calculate the chi-square test statistic for testing whether a complaint is more likely to receive a full investigation when it has video evidence. For this question, only consider unique complaints with complete information. 

In [100]:
# Check the crosstab for investigation vs video
pd.crosstab(data_uni_complete['Is Full Investigation'],\
            data_uni_complete['Complaint Has Video Evidence'])

Complaint Has Video Evidence,False,True
Is Full Investigation,,
False,44529,584
True,21889,1465


In [101]:
# Create contingency table for further testing
vid_vs_invstgtn = pd.crosstab(data_uni_complete['Is Full Investigation'],\
            data_uni_complete['Complaint Has Video Evidence']).values

In [102]:
# Perform chi^2 test of independence
stats.chi2_contingency(vid_vs_invstgtn)

(1312.0319808202814,
 2.744813207558423e-287,
 1,
 array([[43762.91109586,  1350.08890414],
        [22655.08890414,   698.91109586]]))

* Complaints about stop and frisk have been declining. Use linear regression from the year complaints about stop and frisk peaked through 2016 (inclusive) to predict how many stop and frisk incidents in 2018 will eventually lead to a complaint. For this question, only consider unique complaints with complete information. Remember that the count of complaints must be an integer (round to nearest). 

In [117]:
data_uni_complete['Allegation Description'].unique()

array(['Action', 'Physical force', 'Strip-searched', 'Threat of arrest',
       'Property damaged', 'Premises entered and/or searched',
       'Question and/or stop', 'Demeanor/tone',
       'Refusal to provide name/shield number', 'Word', 'Vehicle stop',
       'Race', 'Gun Drawn', 'Other',
       'Refusal to obtain medical treatment', 'Gun Pointed',
       'Refusal to process civilian complaint', 'Vehicle search',
       'Search (of person)', 'Threat of force (verbal or physical)',
       'Seizure of property', 'Gun fired',
       'Failure to show search warrant',
       'Nightstick as club (incl asp & baton)', 'Threat of summons',
       'Radio as club', 'Improper dissemination of medical info',
       'Retaliatory summons', 'Chokehold', 'Retaliatory arrest',
       'Threat to damage/seize property', 'Vehicle',
       'Hit against inanimate object', 'Other blunt instrument as a club',
       'Frisk', 'Sexual orientation', 'Ethnicity', 'Threat to notify ACS',
       'Pepper spray', '

In [142]:
# Filter complaints about stop and frisk (I assume that you have to stop a person to frisk her
# and that complaint about just stop is not a complaint about stop and frisk
data_frisk = data_uni_complete[data_uni_complete['Allegation Description'].\
                  str.contains('Frisk' or 'frisk')]

In [143]:
# Find a year where n complaints peaked
data_frisk['Received Year'].value_counts()

2011    186
2009    181
2007    172
2010    163
2006    153
2008    151
2012    145
2013    133
2014     90
2005     79
2015     66
2016     48
2004      6
2017      1
Name: Received Year, dtype: int64

In [173]:
# Create a new data frame with number of complaints vs year 
years = data_frisk['Received Year'].value_counts().index
vals = data_frisk['Received Year'].value_counts().values
complaints = pd.DataFrame({'year': years, 'n_complaints': vals})

In [174]:
# Limit the dataframe to a given date range and reset index
complaints = complaints[(complaints.year >= 2011) & (complaints.year < 2017)]
complaints.reset_index(drop=True)
complaints

,n_complaints,year
0,186,2011
6,145,2012
7,133,2013
8,90,2014
10,66,2015
11,48,2016


In [178]:
# Build a model with year as a predictor of a number of stop and frisk complaints
lin_reg = LinearRegression()
lin_reg.fit(complaints.year.values.reshape(-1, 1), complaints.n_complaints)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [187]:
# Predict
lin_reg.predict(2018)

array([-13.38095238])

* Each row in the data set corresponds with a specific allegation. Therefore a particular complaint, designated by 'UniqueComplaintId', may have multiple allegations. Consider only allegations with complete information. Is the presence of a certain type of allegation (i.e. 'Allegation FADO Type') indicative that a complaint will contain multiple allegations? Create indicator variables for whether a complaint contains each type of allegation, and perform a linear regression of the number of allegations per complaint against these indicator variables. What is the maximum coefficient of the linear regression? 

In [188]:
# Create a dataset of all complete allegations
data_complete = data.dropna(axis=0)

In [190]:
# Create indicatgor variables
FADO_indicator = pd.get_dummies(data_complete['Allegation FADO Type'])

In [193]:
# Sanity check
data_complete.shape, FADO_indicator.shape

((202390, 16), (202390, 4))

In [208]:
# Join
FADO_complete = data_complete.join(FADO_indicator)

In [209]:
FADO_complete.head(3)

,DateStamp,UniqueComplaintId,Close Year,Received Year,Borough of Occurrence,Is Full Investigation,Complaint Has Video Evidence,Complaint Filed Mode,Complaint Filed Place,Complaint Contains Stop & Frisk Allegations,Incident Location,Incident Year,Encounter Outcome,Reason For Initial Contact,Allegation FADO Type,Allegation Description,Abuse of Authority,Discourtesy,Force,Offensive Language
0,2017-02-07,6,2006,2006,Brooklyn,False,False,Phone,IAB,False,Street/highway,2006,No Arrest or Summons,PD suspected C/V of violation/crime - street,Discourtesy,Action,0,1,0,0
1,2017-02-07,11,2006,2006,Bronx,False,False,Phone,IAB,False,Street/highway,2006,Arrest,Other,Force,Physical force,0,0,1,0
2,2017-02-07,20,2006,2005,Bronx,False,False,Call Processing System,CCRB,True,Street/highway,2005,No Arrest or Summons,Other,Abuse of Authority,Strip-searched,1,0,0,0


In [210]:
# Create total n of allegations variable
FADO_complete['Allegations_total'] = FADO_complete['Abuse of Authority'] + \
FADO_complete['Discourtesy'] + FADO_complete['Force'] + FADO_complete['Offensive Language']

In [212]:
# Drop unused cols
FADO_complete = FADO_complete[['UniqueComplaintId', 'Allegation FADO Type', \
                               'Abuse of Authority', 'Force', \
              'Discourtesy', 'Offensive Language', 'Allegations_total']]

In [216]:
# Resample data frame by unique id
FADO_rsmpld = FADO_complete.groupby('UniqueComplaintId').sum()

In [242]:
FADO_rsmpld_pt1 = FADO_rsmpld[['Abuse of Authority', 'Force', \
              'Discourtesy', 'Offensive Language']].applymap(lambda x: 1 if x > 0 else 0)

FADO_rsmpld_pt2 = FADO_rsmpld['Allegations_total']

FADO_rsmpld_final = FADO_rsmpld_pt1.join(FADO_rsmpld_pt2)

In [248]:
FADO_rsmpld_final.to_csv('C:\\Users\\Ol\\Documents\\DATA ANALYSIS\\DATA_INCUBATOR\\edit.csv')

In [243]:
# Prepare dataset for performing linear regression
y_fado = FADO_rsmpld_final.Allegations_total.values
X_fado = sm.add_constant(FADO_rsmpld_final.drop('Allegations_total', axis=1).values)

In [244]:
# Perform linear regression
model_fado = sm.OLS(y_fado, X_fado)
result_fado = model_fado.fit()
result_fado.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.411
Model:                            OLS   Adj. R-squared:                  0.411
Method:                 Least Squares   F-statistic:                 1.195e+04
Date:                Sat, 21 Jul 2018   Prob (F-statistic):               0.00
Time:                        08:30:20   Log-Likelihood:            -1.3784e+05
No. Observations:               68467   AIC:                         2.757e+05
Df Residuals:                   68462   BIC:                         2.757e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4305      0.017    -25.038      0.000      -0.464      -0.397
x1             2.5697      0.015    170.529      0.000       2.540       2.599
x2             1.9042      0.015    130.857      0.000       1.876       1.933
x3             1.6070      0.014    112.165      0.000       1.579       1.635
x4             1.4769      0.026     57.062      0.000       1.426       1.528
==============================================================================
Omnibus:                    38950.977   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           490730.027
Skew:                           2.505   Prob(JB):                         0.00
Kurtosis:                      15.121   Cond. No.                         5.26
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [371]:
# Grader needs bigger float precision
f'{result_fado.params[1]:.11f}'

'2.56970090753'

* According to NYC.gov there are approximately 36000 officers in New York. The website additionally lists information on all the precincts in each borough. Consider unique complaints (not necessarily with complete information) from incidents in 2016. Assuming that complaints per capita are proportional to officers per capita in each borough, calculate the average number of officers per precinct in each borough (ignore complaints from outside of NYC). What is the ratio of the highest number of officers per precinct to the lowest number of officers per precinct? 

In [255]:
# Save url for scraping precincts
url = 'https://www1.nyc.gov/site/nypd/bureaus/patrol/precincts-landing.page'

In [270]:
# Download the page
page = urllib.request.urlopen(url).read()

In [271]:
# Create the soup
precinct_soup = BeautifulSoup(page, 'lxml')

In [355]:
# Find indexes of borough name rows
for i, c in enumerate(list(precinct_soup.find_all('tr'))):
    if 'subhead' in str(c):
        print(i, c.text)

1 
Manhattan

24 
Bronx

37 
Brooklyn

61 
Queens

78 
Staten Island



In [356]:
# Find the index of the last row
len(precinct_soup.find_all('tr'))

83

In [357]:
# Check if the last row contains precinct info
list(precinct_soup.find_all('tr'))[-1]

<tr>
<td data-label="Precinct"><a href="/site/nypd/bureaus/patrol/precincts/123rd-precinct.page">123rd Precinct</a></td>
<td data-label="Phone">718-948-9311</td>
<td data-label="Address">116 Main Street</td>
</tr>

In [358]:
# Compute n of precincts per borough
n_precincts = {'mnhttn': 24-1-1, 'brnx': 37-24-1, 'brkln': 61-37-1, \
              'queens': 78-61-1, 'sttn': 83-78-1}

In [359]:
# Sanity check
n_precincts

{'brkln': 23, 'brnx': 12, 'mnhttn': 22, 'queens': 16, 'sttn': 4}

In [410]:
# Create new dataframe with data for incidents from 2016 ignoring outside NYC
data_precinct = data[(data['Borough of Occurrence']!= 'Outside NYC') & \
                     (data['Incident Year']==2016)]

In [411]:
# Save unique complaints only
data_precinct = data_precinct.drop_duplicates('UniqueComplaintId')

In [412]:
# Drop rows with missing borough of occurence
data_precinct = data_precinct[~data_precinct['Borough of Occurrence'].isna()]

In [415]:
# Get n of complaints per borough
data_precinct['Borough of Occurrence'].value_counts()

Brooklyn         1102
Manhattan         862
Bronx             792
Queens            601
Staten Island     156
Name: Borough of Occurrence, dtype: int64

In [416]:
# Get total n of complaints
total_cmplnts = data_precinct['Borough of Occurrence'].value_counts().sum()

In [418]:
# Compute percent of complaints per borough
percent_cmplnts_per_boro = {'brkln': 1102/total_cmplnts,\
                           'mnhttn': 862/total_cmplnts,\
                           'brnx': 792/total_cmplnts,\
                           'queens': 601/total_cmplnts,\
                           'sttn': 156/total_cmplnts}

In [424]:
# Compute n officers per borough
officers_per_boro = {}
for key in percent_cmplnts_per_boro.keys():
    officers_per_boro[key] = 36000*percent_cmplnts_per_boro[key]

In [430]:
# Compute mean n of officers per precinct
n_officers_per_precinct_per_boro = []
for key in officers_per_boro.keys():
    n_officers_per_precinct_per_boro.append(round(officers_per_boro[key] / n_precincts[key]))

In [431]:
max(n_officers_per_precinct_per_boro) /\
min(n_officers_per_precinct_per_boro)

1.7558441558441558